In [1]:
# Name: Ephrem Alemayehu
# ID: UGR/4365/12


In [2]:
import numpy as np
import random

In [3]:
# Grid World Environment
class GridWorld:
    def __init__(self, n, m, start, goal, obstacles):
        self.n = n
        self.m = m
        self.start = start
        self.goal = goal
        self.obstacles = obstacles
        self.grid = np.zeros((n, m))
        for obstacle in obstacles:
            self.grid[obstacle] = -1  # Mark obstacles with -1
        self.grid[goal] = 10  # Mark goal with 10
        
    def is_valid(self, state):
        x, y = state
        if 0 <= x < self.n and 0 <= y < self.m and self.grid[x, y] != -1:
            return True
        return False

    def get_next_state(self, state, action):
        if action == "up":
            next_state = (state[0] - 1, state[1])
        elif action == "down":
            next_state = (state[0] + 1, state[1])
        elif action == "left":
            next_state = (state[0], state[1] - 1)
        elif action == "right":
            next_state = (state[0], state[1] + 1)
        else:
            next_state = state
        
        if self.is_valid(next_state):
            return next_state
        else:
            return state
    
    def get_reward(self, state):
        if state == self.goal:
            return 10
        elif self.grid[state] == -1:
            return -10
        else:
            return -1

# Example setup for GridWorld
n, m = 4, 4
start = (0, 0)
goal = (3, 3)
obstacles = [(1, 1), (1, 2), (2, 1)]

grid_world = GridWorld(n, m, start, goal, obstacles)

In [4]:
# Value Iteration
def value_iteration(grid_world, gamma=0.9, theta=1e-6):
    V = np.zeros((grid_world.n, grid_world.m))
    policy = np.zeros((grid_world.n, grid_world.m), dtype=str)
    actions = ["up", "down", "left", "right"]

    while True:
        delta = 0
        for i in range(grid_world.n):
            for j in range(grid_world.m):
                state = (i, j)
                if state == grid_world.goal or grid_world.grid[state] == -1:
                    continue
                v = V[state]
                max_value = float('-inf')
                best_action = None
                for action in actions:
                    next_state = grid_world.get_next_state(state, action)
                    reward = grid_world.get_reward(next_state)
                    value = reward + gamma * V[next_state]
                    if value > max_value:
                        max_value = value
                        best_action = action
                V[state] = max_value
                policy[state] = best_action
                delta = max(delta, abs(v - V[state]))
        if delta < theta:
            break

    return V, policy

V, policy = value_iteration(grid_world)
print("Value Function (Value Iteration):\n", V)
print("Policy (Value Iteration):\n", policy)

Value Function (Value Iteration):
 [[ 1.8098  3.122   4.58    6.2   ]
 [ 3.122   0.      0.      8.    ]
 [ 4.58    0.      8.     10.    ]
 [ 6.2     8.     10.      0.    ]]
Policy (Value Iteration):
 [['d' 'r' 'r' 'd']
 ['d' '' '' 'd']
 ['d' '' 'd' 'd']
 ['r' 'r' 'r' '']]


In [5]:
# Policy Iteration
def policy_evaluation(policy, grid_world, gamma=0.9, theta=1e-6):
    V = np.zeros((grid_world.n, grid_world.m))
    while True:
        delta = 0
        for i in range(grid_world.n):
            for j in range(grid_world.m):
                state = (i, j)
                if state == grid_world.goal or grid_world.grid[state] == -1:
                    continue
                v = V[state]
                action = policy[state]
                next_state = grid_world.get_next_state(state, action)
                reward = grid_world.get_reward(next_state)
                V[state] = reward + gamma * V[next_state]
                delta = max(delta, abs(v - V[state]))
        if delta < theta:
            break
    return V

In [6]:
def policy_improvement(V, grid_world, gamma=0.9):
    policy = np.zeros((grid_world.n, grid_world.m), dtype=str)
    actions = ["up", "down", "left", "right"]
    for i in range(grid_world.n):
        for j in range(grid_world.m):
            state = (i, j)
            if state == grid_world.goal or grid_world.grid[state] == -1:
                continue
            max_value = float('-inf')
            best_action = None
            for action in actions:
                next_state = grid_world.get_next_state(state, action)
                reward = grid_world.get_reward(next_state)
                value = reward + gamma * V[next_state]
                if value > max_value:
                    max_value = value
                    best_action = action
            policy[state] = best_action
    return policy

def policy_iteration(grid_world, gamma=0.9):
    policy = np.random.choice(["up", "down", "left", "right"], size=(grid_world.n, grid_world.m))
    while True:
        V = policy_evaluation(policy, grid_world, gamma)
        new_policy = policy_improvement(V, grid_world, gamma)
        if (new_policy == policy).all():
            break
        policy = new_policy
    return V, policy

V_policy, policy_policy = policy_iteration(grid_world)
print("Value Function (Policy Iteration):\n", V_policy)
print("Policy (Policy Iteration):\n", policy_policy)

Value Function (Policy Iteration):
 [[-9.99999179 -9.99999179 -9.99999179 -9.99999179]
 [-9.99999179  0.          0.         -9.99999179]
 [-9.99999179  0.         -9.99999179 -9.99999179]
 [-9.99999179 -9.99999179 -9.99999179  0.        ]]
Policy (Policy Iteration):
 [['u' 'u' 'u' 'u']
 ['u' '' '' 'u']
 ['u' '' 'u' 'd']
 ['u' 'u' 'r' '']]


In [7]:
# Q-Learning
def q_learning(grid_world, alpha=0.1, gamma=0.9, epsilon=0.1, episodes=1000):
    Q = np.zeros((grid_world.n, grid_world.m, 4))
    actions = ["up", "down", "left", "right"]
    action_indices = {a: i for i, a in enumerate(actions)}

    for _ in range(episodes):
        state = grid_world.start
        while state != grid_world.goal:
            if random.uniform(0, 1) < epsilon:
                action = random.choice(actions)
            else:
                action = actions[np.argmax(Q[state[0], state[1]])]
            next_state = grid_world.get_next_state(state, action)
            reward = grid_world.get_reward(next_state)
            best_next_action = np.argmax(Q[next_state[0], next_state[1]])
            td_target = reward + gamma * Q[next_state[0], next_state[1], best_next_action]
            td_error = td_target - Q[state[0], state[1], action_indices[action]]
            Q[state[0], state[1], action_indices[action]] += alpha * td_error
            state = next_state

    policy = np.zeros((grid_world.n, grid_world.m), dtype=str)
    for i in range(grid_world.n):
        for j in range(grid_world.m):
            best_action_idx = np.argmax(Q[i, j])
            policy[i, j] = actions[best_action_idx]
    return Q, policy

Q, policy_q_learning = q_learning(grid_world)
print("Q-Values (Q-Learning):\n", Q)
print("Policy (Q-Learning):\n", policy_q_learning)

Q-Values (Q-Learning):
 [[[ 0.20231665 -0.86826708  0.41113643  1.8098    ]
  [ 1.50965301  1.6531591   0.46231559  3.122     ]
  [ 2.82027661  2.88370545  1.52829271  4.58      ]
  [ 3.55299774  6.2         2.89960885  3.49792067]]

 [[-2.02835574  1.19628643 -1.89610968 -1.80533789]
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]
  [ 4.49244171  8.          6.06083156  5.85871876]]

 [[-1.08599338  3.34259988 -1.30793839 -1.20479159]
  [ 0.          0.          0.          0.        ]
  [-0.1         0.69521912 -0.1         7.65943287]
  [ 5.69713239 10.          4.9361339   7.70025496]]

 [[-0.54165715 -0.58519851  0.29891246  5.62826773]
  [ 0.95510404 -0.199       0.19996868  7.80594192]
  [ 0.14574014 -0.1        -0.109       9.96956747]
  [ 0.          0.          0.          0.        ]]]
Policy (Q-Learning):
 [['r' 'r' 'r' 'd']
 ['d' 'u' 'u' 'd']
 ['d' 'u' 'r' 'd']
 ['r' 'r' 'r' 'u']]


In [8]:
# Multi-Armed Bandit Environment
class MultiArmedBandit:
    def __init__(self, k, reward_distributions):
        self.k = k
        self.reward_distributions = reward_distributions
        self.q_true = [np.mean(dist) for dist in reward_distributions]
        
    def pull(self, arm):
        return np.random.choice(self.reward_distributions[arm])

# Example setup for MultiArmedBandit
k = 10
reward_distributions = [np.random.normal(loc, 1, 1000) for loc in range(1, k+1)]
bandit = MultiArmedBandit(k, reward_distributions)

# UCB Algorithm for Multi-Armed Bandit
def ucb(bandit, N=1000, c=2):
    Q = np.zeros(bandit.k)
    N_a = np.zeros(bandit.k)
    total_reward = 0

    for t in range(1, N+1):
        ucb_values = Q + c * np.sqrt(np.log(t) / (N_a + 1e-5))
        arm = np.argmax(ucb_values)
        reward = bandit.pull(arm)
        N_a[arm] += 1
        Q[arm] += (reward - Q[arm]) / N_a[arm]
        total_reward += reward

    return Q, total_reward

Q_ucb, total_reward_ucb = ucb(bandit)
print("Q-Values (UCB):\n", Q_ucb)
print("Total Reward (UCB):\n", total_reward_ucb)

Q-Values (UCB):
 [ 2.18530386  2.23517113  4.6713612   3.49230406  3.9093488   4.8336039
  7.04370862  7.89710713  9.0669796  10.05967864]
Total Reward (UCB):
 9966.208897800314


In [9]:
# Evaluation
def evaluate_bandit(agent, bandit, N=1000):
    total_reward = 0
    for _ in range(N):
        action = agent(bandit)
        reward = bandit.pull(action)
        total_reward += reward
    return total_reward

# Example optimal strategy comparison
def optimal_strategy(bandit):
    return np.argmax(bandit.q_true)

In [10]:
total_reward_optimal = evaluate_bandit(optimal_strategy, bandit)
print("Total Reward (Optimal Strategy):\n", total_reward_optimal)
print("Total Reward (UCB):\n", total_reward_ucb)

Total Reward (Optimal Strategy):
 9992.865927120662
Total Reward (UCB):
 9966.208897800314
